In [1]:
!which python

/Users/apabook/Desktop/ds-deployment/venv/bin/python


In [2]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [3]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Wallets setup

In [6]:
url = data_space_provider + "/api/v1/wallet/onboard"
response = requests.request("POST", url)
response

<Response [201]>

In [7]:
url = data_space_consumer + "/api/v1/wallet/onboard"
response = requests.request("POST", url)
response

<Response [201]>

## Participants onboarding

### Current Participants information

In [8]:
url = data_space_provider + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "participant_slug": "Myself",
    "participant_type": "Provider",
    "base_url": "http://127.0.0.1:1203",
    "token": null,
    "saved_at": "2025-10-07T10:38:17.860264",
    "last_interaction": "2025-10-07T13:51:06.286954",
    "is_me": true
}


In [9]:
url = data_space_consumer + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
    "participant_slug": "Myself",
    "participant_type": "Consumer",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "saved_at": "2025-10-07T13:51:12.325737",
    "last_interaction": "2025-10-07T13:51:12.326074",
    "is_me": true
}


### Consumer registering in Provider via Verifiable presentation

In [11]:
url = data_space_provider + "/api/v1/did.json"
response = requests.request("GET", url)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["id"]
data_space_provider_participant_id

'did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ'

In [13]:
url = data_space_consumer + "/api/v1/request/onboard/provider"
payload = {
    "url": data_space_provider + "/api/v1/access",
    "id": data_space_provider_participant_id,
    "slug": "provider",
    "actions": "talk"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2FvkuDcbIaaWho&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fpd%2FvkuDcbIaaWho&client_id_scheme=redirect_uri&nonce=blUBhIF1pTN6&response_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2FvkuDcbIaaWho'

### Provider and Consumer ids

In [27]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Consumer":
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
        "participant_slug": "rainbow_consumer",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": "RVL1BeqGmSPZCo0Tpa_LyGXnbcRv1AR-n0k2Vx7BGwA",
        "saved_at": "2025-10-07T10:39:39.206112",
        "last_interaction": "2025-10-07T10:39:39.206113",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4

In [28]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Provider":
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
        "participant_slug": "provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": "RVL1BeqGmSPZCo0Tpa_LyGXnbcRv1AR-n0k2Vx7BGwA",
        "saved_at": "2025-10-07T10:39:39.213457",
        "last_interaction": "2025-10-07T10:39:39.213458",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJo

## Create Catalog instances and metadata

In [29]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:549667ee-9eae-4038-bf5f-80ee76bdce3a'

In [30]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2'

In [31]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:c2a5990c-17ba-4ccc-811c-3b7fbdac616b'

In [32]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

'urn:uuid:87faebe5-ca37-4649-8d8b-ed5a41621ddd'

## Create policies

In [33]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:b2277139-3c08-4a4f-a6b9-7e1155f3136b",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2"
}


## Contract Negotiation

In [36]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "offer": {
    "@id": "urn:uuid:b2277139-3c08-4a4f-a6b9-7e1155f3136b",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
    "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
    "state": "REQUESTED"
  }
}


In [37]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "offer": {
    "@id": "urn:uuid:b2277139-3c08-4a4f-a6b9-7e1155f3136b",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    

In [38]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "offer": {
    "@id": "urn:uuid:b2277139-3c08-4a4f-a6b9-7e1155f3136b",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
    "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
    "state": "REQUESTED"
  }
}


In [39]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "offer": {
    "@id": "urn:uuid:b2277139-3c08-4a4f-a6b9-7e1155f3136b",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    

In [40]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
    "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
    "state": "ACCEPTED"
  }
}


In [41]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "agreement": {
    "@id": "urn:uuid:861dea6f-bb78-4cb5-99df-24d1b541cf75",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:uuid:4880bfa7-37f9-48e8-a225-397c6bbc3ed2",
    "assigner": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "assignee": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5T

In [42]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
    "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
    "state": "VERIFIED"
  }
}


In [43]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
  "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4ee52c75-d42a-4b3f-a213-f08858a12040",
    "consumerPid": "urn:uuid:9efd5a0a-b99e-4e35-8a80-0235cce2a4dd",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [44]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
  "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "agreementId": "urn:uuid:861dea6f-bb78-4cb5-99df-24d1b541cf75",
  "callbackAddress": "http://127.0.0.1:1100/urn:uuid:b0fbc4c3-9754-4e44-b9e7-2e0819bcc57c",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "state": "REQUESTED"
  }
}


In [45]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
  "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://127.0.0.1:1200/data/urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "state": "STARTED"
  }
}


In [46]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://127.0.0.1:1100/data/urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-10-07T14:18:40.109051",
  "updated_at": "2025-10-07T14:18:40.177676",
  "state": "STARTED"
}


In [47]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)


     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [48]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
  "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "state": "SUSPENDED"
  }
}


In [49]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

HTTP Error 403: Forbidden


In [50]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
  "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "state": "STARTED"
  }
}


In [51]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [52]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
  "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "state": "SUSPENDED"
  }
}


In [53]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
  "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:dfae753b-b3bf-4ce6-a732-fcee442328bf",
    "consumerPid": "urn:uuid:5606a90f-0519-4bb0-b425-50d7f2aa5c10",
    "state": "COMPLETED"
  }
}
